In [8]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
sys.path.append('../')
import json
from lib.TabularDataset import dataset_params
import copy
import torch

res_dir = Path('/scratch/hdd001/home/haoran/explanations-subpopulations/')

In [9]:
df_raw = []
for path in res_dir.glob('**/*.pkl'):
    args = json.load((path.parent/'args.json').open('r'))
    res = torch.load(path)
    for i in ['WHITE', 'race1']:
        if i in res:
            res['Race'] = res[i]
            del res[i]
    for i in ['MALE', 'male']:
        if i in res:
            res['Gender'] = res[i]
            del res[i]        
            
    df_raw.append({**args, **res})
    
df = pd.DataFrame(df_raw).drop(columns = ['pca_dim', 'seed', 'wass_eps', 'experiment'])

In [10]:
def expand(x):
    ret = {}
    for i, j in x['Gender'].iloc[0].items():
        ret[('Gender', i)] = j
    for i, j in x['Race'].iloc[0].items():
        ret[('Race', i)] = j
    return pd.Series(ret)   

df2 = (df.groupby(['dataset', 'dist_method', 'fairness_type'])
     .apply(lambda x: expand(x)).to_frame()
     .reset_index().rename(columns = {'level_3': 'attr', 'level_4': 'grp', 0: 'dist'}))

In [11]:
final_df = df2.pivot_table(values = 'dist', index = ['dataset', 'dist_method', 'fairness_type'], columns = ['attr', 'grp'], aggfunc = lambda x: x)

In [12]:
final_df.applymap("{0:.3f}".format)

attr                                  Gender             Race                  \
grp                                      0.0     1.0      0.0     1.0     2.0   
dataset    dist_method fairness_type                                            
adult      clf         dp              0.933   0.933    0.691   0.950   0.766   
                       eo_n            0.912   0.912    0.682   0.928   0.762   
                       eo_p            0.980   0.980    0.831   0.977   0.720   
           mmd         dp              0.004   0.001    0.006   0.031   0.003   
                       eo_n            0.003   0.001    0.007   0.030   0.002   
                       eo_p            0.015   0.001    0.037   0.043   0.007   
           pca_mmd     dp              0.016   0.004    0.007   0.043   0.009   
                       eo_n            0.011   0.004    0.007   0.038   0.006   
                       eo_p            0.057   0.002    0.037   0.077   0.007   
           pca_wass    dp              2.566   0.391    1.800   8.344   1.171   
                       eo_n            2.200   0.507    2.009   7.764   1.160   
                       eo_p            6.580   0.681    4.333  13.182   1.622   
           wass        eo_p           48.019   5.428  109.455  47.233  70.009   
lsac       clf         dp              0.630   0.630    0.814   0.738   0.689   
                       eo_n            0.566   0.566    0.853   0.553   0.476   
                       eo_p            0.628   0.628    0.799   0.754   0.695   
           mmd         dp              0.000   0.000    0.000   0.011   0.006   
                       eo_n            0.002   0.001    0.004   0.014   0.020   
                       eo_p            0.000   0.000    0.000   0.011   0.006   
           pca_mmd     dp              0.000   0.000    0.001   0.015   0.009   
                       eo_n            0.002   0.001    0.005   0.014   0.019   
                       eo_p            0.000   0.000    0.000   0.014   0.009   
           pca_wass    dp              0.187   0.173    0.251   1.668   1.253   
                       eo_n            0.498   0.383    1.009   2.083   2.843   
                       eo_p            0.188   0.173    0.226   1.591   1.289   
           wass        dp              0.628   0.491    0.384   3.709   2.788   
                       eo_n            1.780   1.470    2.065   6.419   6.344   
                       eo_p            0.645   0.499    0.335   3.689   3.036   
mimic_tab  clf         dp              0.687   0.687    0.714   0.574   0.610   
                       eo_n            0.696   0.696    0.731   0.567   0.614   
                       eo_p            0.630   0.630    0.610   0.593   0.545   
           mmd         dp              0.000   0.000    0.002   0.000   0.000   
                       eo_n            0.000   0.000    0.003   0.000   0.000   
                       eo_p            0.001   0.001    0.007   0.002   0.000   
           pca_mmd     dp              0.000   0.000    0.004   0.000   0.000   
                       eo_n            0.000   0.000    0.004   0.000   0.000   
                       eo_p            0.001   0.001    0.006   0.002   0.000   
           pca_wass    dp              0.589   0.478    4.269   0.914   0.320   
                       eo_n            0.615   0.455    4.368   0.980   0.328   
                       eo_p            1.537   1.278    5.413   2.432   0.792   
           wass        eo_p           15.499  13.877   44.359  28.362   6.703   
recidivism clf         dp              0.740   0.740    0.667   0.667     nan   
                       eo_n            0.748   0.748    0.654   0.654     nan   
                       eo_p            0.639   0.639    0.640   0.640     nan   
           mmd         dp              0.006   0.000    0.000   0.000     nan   
                       eo_n            0.006   0.000    0.001   0.000     nan   
                       eo_p    

In [13]:
final_df.to_pickle('dataset_dist.pkl')